In [1]:
"""
    Who: Ericson Mattoso
    When: 04/nov/2021
    What: This script is responsable calculate houses priority
    Why: This calculation will help users to find a best deal house
    Where: data comes from previous dataframes. 
    How: calculates based a weighted variables
    todo: moving processed files that should be on raw
"""

# libs
from bs4 import BeautifulSoup
from datetime import date
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import time
today = date.today()

# read data
df_pararius = pd.read_csv(
    '../data/processed/df_coo_pararius.csv', index_col=[0])
house_temp = pd.read_csv('../data/raw/house_temp.csv', index_col=[0])

# get unique urls in each dataframe
temp1 = df_pararius['url'].unique()
temp2 = house_temp['url'].unique()

# get only new links that are not saved on our database
urls = list(set(temp1) - set(temp2))

# print how many new urls we have now
# This is the number of links we need to scrape
print(len(urls))


def key_val(text):
    part = html_soup.find("dt", text=text)
    key = (part.text.strip())
    val = (part.findNext("dd").text.strip())
    return key, val


def scrape_web(url):
    """
        the purpose of the function is to scrape a page
        it creates fake browser to do not getting blocked
        then it transform the html into a string value
    """
    chrome_options = Options()
    chrome_options.add_argument("--window-size=%s" % "1,1")
    driver = webdriver.Chrome('temp/chromedriver', chrome_options=chrome_options)
    driver.set_window_position(-10000, 0)
    driver.get(url)
    html = driver.page_source
    html_soup = BeautifulSoup(html)
    driver.close()
    return html_soup, 'hdr'

not_available = []
description_2 = []
# This is going in each page to save the dates of
i=0

# urls = ['https://www.pararius.com/apartment-for-rent/rotterdam/88175c6a/middellandplein','https://www.pararius.com/apartment-for-rent/amsterdam/e814cfb4/sumatrastraat']

for url in urls:
    # flag
    print(">>>>>>>>>", i, url)
    description_1 = {}
    try:
        html_soup, hdr = scrape_web(url)
        key1, val1 = key_val("Offered since")
        key2, val2 = key_val("Available")
        description_1.update({
            'url': url,
            key1: val1,
            key2: val2,
        })
        description_2.append(description_1)
        i = i+1
        print('done')
        print()
    except:
        print('error')
        pass
house_temp_ = pd.DataFrame(description_2)
house_temp = pd.concat([house_temp, house_temp_], 0)

def fixing_time_delta(df, time, my_ofset):
    fixing_time_delta = df.loc[df['Offered since'].str.contains(
        time, na=False), 'Offered since']
    fixing_time_delta = fixing_time_delta.str.replace(
        "\D", "", regex=True).astype(int)
    fixing_time_delta = today - fixing_time_delta.apply(my_ofset)
    fixing_time_delta = pd.to_datetime(
        fixing_time_delta).dt.strftime('%d-%m-%Y')
    df.loc[df['Offered since'].str.contains(
        time, na=False), 'Offered since'] = fixing_time_delta
    return df


try:
    house_temp = fixing_time_delta(house_temp, "week", pd.offsets.Week)
    house_temp = fixing_time_delta(house_temp, "month", pd.offsets.MonthBegin)
except:
    print('erro')
    pass

today = date.today()
house_temp['Available'] = house_temp['Available'].str.replace("From", "")
house_temp['Available'] = house_temp['Available'].str.replace(
    "Immediately|In consultation", str(today))
house_temp['Available'] = pd.to_datetime(
    house_temp['Available']).dt.strftime('%d-%m-%Y')
house_temp = house_temp.drop_duplicates(subset=['url']).reset_index(drop=True)
result = pd.merge(df_pararius, house_temp, how='left', on='url')

1466
>>>>>>>>> 0 https://www.pararius.com/house-for-rent/nijmegen/b0ba9176/lange-hezelstraat


<ipython-input-1-07621a4d9800>:57: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('temp/chromedriver', chrome_options=chrome_options)


done

>>>>>>>>> 1 https://www.pararius.com/apartment-for-rent/leiden/45840d3a/virulypad
done

>>>>>>>>> 2 https://www.pararius.com/apartment-for-rent/haarlem/8d3e6446/engelenburg
done

>>>>>>>>> 3 https://www.pararius.com/studio-for-rent/zwolle/83e3d88d/venestraat
done

>>>>>>>>> 4 https://www.pararius.com/apartment-for-rent/velp-gld/55a37f6c/aalscholversingel
done

>>>>>>>>> 5 https://www.pararius.com/apartment-for-rent/utrecht/b0a4df46/zeeltstraat
done

>>>>>>>>> 6 https://www.pararius.com/apartment-for-rent/groningen/a92972f7/boterdiep
done

>>>>>>>>> 7 https://www.pararius.com/apartment-for-rent/amstelveen/cd63ca3b/logger
done

>>>>>>>>> 8 https://www.pararius.com/apartment-for-rent/breda/357ec236/mgr-horsthuisstraat
done

>>>>>>>>> 9 https://www.pararius.com/apartment-for-rent/amsterdam/8124eaa9/ijselstraat
done

>>>>>>>>> 10 https://www.pararius.com/apartment-for-rent/breda/051af608/hertzogstraat
done

>>>>>>>>> 11 https://www.pararius.com/house-for-rent/delft/74b56f97/hofzoom
do

done

>>>>>>>>> 88 https://www.pararius.com/apartment-for-rent/amsterdam/775a8f6b/van-nijenrodeweg
done

>>>>>>>>> 89 https://www.pararius.com/apartment-for-rent/amsterdam/ee491fdd/cruquiusweg
done

>>>>>>>>> 90 https://www.pararius.com/apartment-for-rent/rotterdam/cb5c2994/mariniersweg
done

>>>>>>>>> 91 https://www.pararius.com/apartment-for-rent/alkmaar/9d41fec9/tuinderspad
done

>>>>>>>>> 92 https://www.pararius.com/apartment-for-rent/vlissingen/6f90487a/vrouwestraat
done

>>>>>>>>> 93 https://www.pararius.com/apartment-for-rent/waalre/9279bedc/canisiushof
done

>>>>>>>>> 94 https://www.pararius.com/apartment-for-rent/amsterdam/8b9a0d35/rombout-hogerbeetsstraat
done

>>>>>>>>> 95 https://www.pararius.com/apartment-for-rent/rotterdam/24ba2d3a/binnenrotte
done

>>>>>>>>> 96 https://www.pararius.com/apartment-for-rent/amsterdam/db874e5d/eerste-schinkelstraat
done

>>>>>>>>> 97 https://www.pararius.com/room-for-rent/maastricht/d85e30fa/frankenstraat
done

>>>>>>>>> 98 https://www.parar

done

>>>>>>>>> 173 https://www.pararius.com/apartment-for-rent/geldrop/74c81f84/de-raadstraat
done

>>>>>>>>> 174 https://www.pararius.com/apartment-for-rent/schiedam/8900bbce/huysmansstraat
done

>>>>>>>>> 175 https://www.pararius.com/apartment-for-rent/eindhoven/23bd7d08/zwembadweg
done

>>>>>>>>> 176 https://www.pararius.com/house-for-rent/amstelveen/5c7b5998/hendrik-van-borsselenkade
done

>>>>>>>>> 177 https://www.pararius.com/apartment-for-rent/amsterdam/57309a18/vrouwengelukhof
done

>>>>>>>>> 178 https://www.pararius.com/apartment-for-rent/rotterdam/929d481d/eendrachtsstraat
done

>>>>>>>>> 179 https://www.pararius.com/house-for-rent/groningen/0f6c448c/zwarteweg
done

>>>>>>>>> 180 https://www.pararius.com/apartment-for-rent/amsterdam/77e08d37/eerste-jan-van-der-heijdenstraat
done

>>>>>>>>> 181 https://www.pararius.com/apartment-for-rent/schiedam/23d2fb64/van-beverenstraat
done

>>>>>>>>> 182 https://www.pararius.com/apartment-for-rent/den-haag/78b9318a/herderinnestraat
done


...

done

>>>>>>>>> 1188 https://www.pararius.com/house-for-rent/almere/e6ca6c11/hennahof
done

>>>>>>>>> 1189 https://www.pararius.com/apartment-for-rent/den-haag/a41a56f7/bussumsestraat
done

>>>>>>>>> 1190 https://www.pararius.com/apartment-for-rent/den-haag/539bc712/laan-van-wateringse-veld
done

>>>>>>>>> 1191 https://www.pararius.com/apartment-for-rent/arnhem/e2ba5943/rosendaalsestraat
done

>>>>>>>>> 1192 https://www.pararius.com/apartment-for-rent/rotterdam/0ef7568b/slaghekstraat
done

>>>>>>>>> 1193 https://www.pararius.com/apartment-for-rent/eindhoven/5429802e/reinkenstraat
done

>>>>>>>>> 1194 https://www.pararius.com/house-for-rent/venlo/f8375aee/neptunusstraat
done

>>>>>>>>> 1195 https://www.pararius.com/apartment-for-rent/rotterdam/0319603a/walchersestraat
done

>>>>>>>>> 1196 https://www.pararius.com/house-for-rent/aalsmeer/42135bb6/helling
done

>>>>>>>>> 1197 https://www.pararius.com/apartment-for-rent/breukeleveen/a05e426a/herenweg
done

>>>>>>>>> 1198 https://www.parari

done

>>>>>>>>> 1272 https://www.pararius.com/apartment-for-rent/eindhoven/4a9db482/hertog-hendrik-van-brabantplein
done

>>>>>>>>> 1273 https://www.pararius.com/apartment-for-rent/amsterdam/37c70357/onbekendegracht
done

>>>>>>>>> 1274 https://www.pararius.com/apartment-for-rent/amsterdam/3f9cf452/burgemeester-hogguerstraat
done

>>>>>>>>> 1275 https://www.pararius.com/apartment-for-rent/amersfoort/3a400157/stadsring
done

>>>>>>>>> 1276 https://www.pararius.com/studio-for-rent/tilburg/7ac050c1/gluckstraat
done

>>>>>>>>> 1277 https://www.pararius.com/apartment-for-rent/den-haag/f109a63e/laan-van-nieuw-oost-indie
done

>>>>>>>>> 1278 https://www.pararius.com/house-for-rent/arnhem/ecf44753/helenastraat
done

>>>>>>>>> 1279 https://www.pararius.com/apartment-for-rent/amsterdam/f5c1dd96/marcusstraat
done

>>>>>>>>> 1280 https://www.pararius.com/apartment-for-rent/haarlem/05ee6677/kamperlaan
done

>>>>>>>>> 1281 https://www.pararius.com/apartment-for-rent/rotterdam/682f29df/westzeedijk
do

done

>>>>>>>>> 1354 https://www.pararius.com/house-for-rent/almere/4fce3f19/groep-werninkhof
done

>>>>>>>>> 1355 https://www.pararius.com/apartment-for-rent/broek-in-waterland/51bf2bb3/overlekergouw
done

>>>>>>>>> 1356 https://www.pararius.com/apartment-for-rent/amsterdam/e75ccdaa/montelbaanstraat
done

>>>>>>>>> 1357 https://www.pararius.com/house-for-rent/maastricht/96aacb0d/sint-teunisstraat
done

>>>>>>>>> 1358 https://www.pararius.com/apartment-for-rent/vlissingen/03541897/coosje-buskenstraat
done

>>>>>>>>> 1359 https://www.pararius.com/apartment-for-rent/den-haag/454699b8/burgemeester-van-der-werffstraat
done

>>>>>>>>> 1360 https://www.pararius.com/house-for-rent/poortugaal/fc9d2e62/burgemeester-breebaartlaan
done

>>>>>>>>> 1361 https://www.pararius.com/house-for-rent/utrecht/67d65413/2e-delistraat
done

>>>>>>>>> 1362 https://www.pararius.com/apartment-for-rent/amersfoort/2cbb4088/oliesteeg
done

>>>>>>>>> 1363 https://www.pararius.com/apartment-for-rent/ede/1a5b008c/de-ha

done

>>>>>>>>> 1436 https://www.pararius.com/apartment-for-rent/noordwijk-zh/a52c18d3/springtij
done

>>>>>>>>> 1437 https://www.pararius.com/apartment-for-rent/den-haag/16b0652b/stalpertstraat
done

>>>>>>>>> 1438 https://www.pararius.com/apartment-for-rent/groningen/48fc209b/westerhavenstraat
done

>>>>>>>>> 1439 https://www.pararius.com/apartment-for-rent/rotterdam/efd3cd83/ijsclubstraat
done

>>>>>>>>> 1440 https://www.pararius.com/apartment-for-rent/den-haag/f834e930/koningin-emmakade
done

>>>>>>>>> 1441 https://www.pararius.com/apartment-for-rent/den-haag/d1751f81/hobbemastraat
done

>>>>>>>>> 1442 https://www.pararius.com/apartment-for-rent/eindhoven/d3ea6f71/volderhof
done

>>>>>>>>> 1443 https://www.pararius.com/apartment-for-rent/purmerend/f978567c/nijlstraat
done

>>>>>>>>> 1444 https://www.pararius.com/apartment-for-rent/breda/9a09696f/boeimeerlaan
done

>>>>>>>>> 1445 https://www.pararius.com/apartment-for-rent/amsterdam/ef711123/derde-oosterparkstraat
done

>>>>>>>>> 14

<ipython-input-1-07621a4d9800>:93: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  house_temp = pd.concat([house_temp, house_temp_], 0)
<ipython-input-1-07621a4d9800>:117: FutureWarning: The default value of regex will change from True to False in a future version.
  house_temp['Available'] = house_temp['Available'].str.replace(


In [6]:

result2 = result[[
    'deal',
    'garden-surface-area',
    'img',
    'price',
    'link',
    'agency',
    'surface-area',
    'interior',
    'latitude',
    'longitude',
    'number-of-rooms',
    'Plaats',
    'Provincie',
    'url',
    'date',
    'status',
    'irl',
    'image',
    'address',
    'street',
    'Offered since',
    'Available',
]]
result2 = result2.rename(columns={
    'img': 'Add',
    'price': 'Price',
    'Plaats': 'City',
    'surface-area': 'Area',
    'number-of-rooms': 'Rooms',
    'garden-surface-area': 'Garden',
    'Offered since': 'Offered',
    'Available': 'Available', })


In [7]:
result2.to_csv('../data/processed/df_coo_pararius.csv')
house_temp.to_csv('../data/raw/house_temp.csv')

In [5]:
result.columns

Index(['price', 'address', 'street', 'agency', 'irl', 'image', 'surface-area',
       'status', 'number-of-rooms', 'interior', 'construction-period',
       'plot-size', 'garden-surface-area', 'postcode', 'date', 'latitude',
       'longitude', 'Straat', 'Netnummer', 'Postcodenummer', 'Buurt', 'Wijk',
       'Plaats', 'Gemeente', 'Provincie', 'Straten', 'url', 'link', 'img',
       'train', 'deal', 'Offered since', 'Available'],
      dtype='object')